# Projeto 1 - Ciência dos Dados

Nome: Bruno Boldrim Saboya

Nome: Murilo Prado Weyne

**Atenção:** Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import re 
import string

**Em `filename`, coloque o nome do seu arquivo de dados!**

In [4]:
import os

filename = 'XBOX.xlsx'
if filename in os.listdir():
    print(f'Encontrei o arquivo {filename}, tudo certo para prosseguir com a prova!')
else:
    print(f'Não encontrei o arquivo {filename} aqui no diretório {os.getcwd()}, será que você não baixou o arquivo?')

Encontrei o arquivo XBOX.xlsx, tudo certo para prosseguir com a prova!


In [44]:

def cleanup(text):
    limpar = ['@', '(', ')', '[', ']', '+', '-', '\n', '\t']
    punctuation = '[!-.:?;\/]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, '', text)
    for t in limpar:
        text_subbed = text_subbed.replace(t, '')
    text_subbed=text_subbed.replace('ã', 'a')
    text_subbed=text_subbed.replace('â', 'a')
    text_subbed=text_subbed.replace('á', 'a')
    text_subbed=text_subbed.replace('à', 'a')
    text_subbed=text_subbed.replace('é', 'e')
    text_subbed=text_subbed.replace('è', 'e')
    text_subbed=text_subbed.replace('ê', 'e')
    text_subbed=text_subbed.replace('î', 'i')
    text_subbed=text_subbed.replace('í', 'i')
    text_subbed=text_subbed.replace('ì', 'i')
    text_subbed=text_subbed.replace('ó', 'o')
    text_subbed=text_subbed.replace('ò', 'o')
    text_subbed=text_subbed.replace('õ', 'o')
    text_subbed=text_subbed.replace('ô', 'o')
    text_subbed=text_subbed.replace('ú', 'u')
    text_subbed=text_subbed.replace('ù', 'a')
    text_subbed=text_subbed.replace('tbm', 'tambem')    
    text_subbed=text_subbed.replace('tb', 'tambem')
    text_subbed=text_subbed.replace('vc', 'voce')
    text_subbed=text_subbed.replace('nois', 'nos')
    text_subbed=text_subbed.replace('gnt', 'gente')
    return text_subbed

Carregando a base de dados com os tweets classificados como relevantes e não relevantes:

In [6]:
train = pd.read_excel(filename)
train.head(5)

,Treinamento,Relevancia
0,@mr_garcez tava em dúvida do xbox one é\ne o ps3,1
1,@sonystasensato @italotravassoss @fuck_off_mat...,1
2,rt @capitaocritica: então galera hoje quê é o ...,1
3,as pré-vendas do xbox series s e xbox series x...,1
4,rt @sbtonline: mais 5 jogos vão entrar para o ...,1


In [7]:
test = pd.read_excel(filename, sheet_name = 'Teste')
test.head(5)

,Teste
0,@drakesincero1 se sair esta parceira o xbox va...
1,@verligameplay @guisouzadias comecei nesse aqu...
2,rt @kah_xp: 900 jogos concluídos 100% no xbox!...
3,hoje é dia de xbox mil grau chorar no twitter ...
4,@rebekah_soaress me add no xbox? ou omlet pra ...


___
## Classificador automático de sentimento


Faça aqui uma descrição do seu produto e o que considerou como relevante ou não relevante na classificação dos tweets.

Consideramos relevantes os tweets que falam sobre o produto e suas características e não relevantes aqueles que não falam do produto e/ou citam o nome sem falar das características.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [11]:
#Primeiro, precisamos definir uma varíavel categórica
train.loc[:,'Relevancia'] = train['Relevancia'].astype('category')
train['Relevancia'].cat.categories = ["Irrelevante", "Relevante"]
train

,Treinamento,Relevancia
0,@mr_garcez tava em dúvida do xbox one é\ne o ps3,Relevante
1,@sonystasensato @italotravassoss @fuck_off_mat...,Relevante
2,rt @capitaocritica: então galera hoje quê é o ...,Relevante
3,as pré-vendas do xbox series s e xbox series x...,Relevante
4,rt @sbtonline: mais 5 jogos vão entrar para o ...,Relevante
...,...,...
295,@renansimeonoff1 @xgrey_wolf93 a maior propaga...,Irrelevante
296,"@murilo_beeble rapaz, ver youtube ficou bom de...",Irrelevante
297,rt @bru_leonel1986: @renansimeonoff1 @xgrey_wo...,Irrelevante
298,"@luizknight agora sonysta migra para o xbox, j...",Irrelevante


In [14]:
#Agora, vamos separar os tweets Relevantes dos Irrelevantes, para fazer uma melhor análise.
relevantes = train['Relevancia'] == 'Relevante'
relevantes = train.loc[relevantes, ['Treinamento']].copy()
relevantes

,Treinamento
0,@mr_garcez tava em dúvida do xbox one é\ne o ps3
1,@sonystasensato @italotravassoss @fuck_off_mat...
2,rt @capitaocritica: então galera hoje quê é o ...
3,as pré-vendas do xbox series s e xbox series x...
4,rt @sbtonline: mais 5 jogos vão entrar para o ...
...,...
289,"@eduoiiveira eu não vi, a única coisa que eu v..."
291,rt @ppggnaarea: tga2019 phill surpreendeu o mu...
292,"rt @xboxbr: pela primeira vez no console, uma ..."
294,@opcxbox @xboxbr já é estratégico...\nquem não...


In [15]:
#Agora, vamos separar os tweets Relevantes dos Irrelevantes, para fazer uma melhor análise.
irrelevantes = train['Relevancia'] == 'Irrelevante'
irrelevantes = train.loc[irrelevantes, ['Treinamento']].copy()
irrelevantes

,Treinamento
6,rt @chief117_com_br: não queria desviar o foco...
7,rt @siathosa: hyper x cloud stinger core verme...
9,rt @xboxbr: a arma mais poderosa da época medi...
10,rt @chief117_com_br: e pelo jeito conseguiram ...
11,"@xkaioh @xboxbr ""ai quem falou merda foi ela""\..."
...,...
293,#osplayers o xbox não aguentou essa geração e ...
295,@renansimeonoff1 @xgrey_wolf93 a maior propaga...
296,"@murilo_beeble rapaz, ver youtube ficou bom de..."
297,rt @bru_leonel1986: @renansimeonoff1 @xgrey_wo...


In [17]:
#Criaremos uma lista contendo todas as palavras relevantes e outra contendo todas as palavras irrelevantes, para passarmos para um arquivo de texto.
lista_relevantes = relevantes["Treinamento"].tolist()

lista_irrelevantes = irrelevantes['Treinamento'].tolist()

In [20]:
#Utilizaremos a função "join" para facilitar a criação de um documento de texto para rodar a função[cleanup] futuramente.
lista_relevantes_txt = ''.join(lista_relevantes)
lista_irrelevantes_txt = ''.join(lista_irrelevantes)

In [25]:
#Finalmente, podemos criar os arquivos de textos para os tweets relevantes e irrelevantes, respectivamente.
#Tendo em mente que temos caracteres com acentos, utilizaremos UTF-8-SIG.
arquivo = open('relevantes.txt', 'w', encoding="utf-8-sig")
arquivo.write(lista_relevantes_txt)
arquivo.close()

arquivo2 = open('irrelevantes.txt', 'w', encoding='utf-8-sig')
arquivo2.write(lista_irrelevantes_txt)
arquivo2.close()

In [29]:
#Agora precisamos abrir os arquivos e novamente usaremos o encoding=utf-8-sig pelo mesmo motivo.
with open('relevantes.txt', 'r', encoding='utf-8-sig') as arq_texto:
    relevante_txt = arq_texto.read()

@mr_garcez tava em dúvida do xbox one é
e o ps3@sonystasensato @italotravassoss @fuck_off_matt "comunidade xbox", os caras se auto elegem representantes. são uns coitados.rt @capitaocritica: então galera hoje quê é o velório do xbox? caixinha chora e mãe não vê? #playstation5 #playstationshowcase https://t.co…as pré-vendas do xbox series s e xbox series x serão limitadas

» https://t.co/8fhwitlps4 htt


In [30]:
#Agora precisamos abrir os arquivos e novamente usaremos o encoding=utf-8-sig pelo mesmo motivo.
with open('irrelevantes.txt', 'r', encoding='utf-8-sig') as arq_texto:
    irrelevante_txt = arq_texto.read()

In [49]:
#Executando a função cleanup nos textos
relevante_txt_clean = cleanup(relevante_txt)

irrelevante_txt_clean = cleanup(irrelevante_txt)

In [52]:
#Vamos assumir agora que, todos as palavras analisadas serão contidas dentro do conjunto [tweets_total]
tweets_total = relevante_txt_clean + irrelevante_txt_clean
tweets_total.split()

['mr_garcez',
 'tava',
 'em',
 'duvida',
 'do',
 'xbox',
 'one',
 'ee',
 'o',
 'ps3sonystasensato',
 'italotravassoss',
 'fuck_off_matt',
 'comunidade',
 'xbox',
 'os',
 'caras',
 'se',
 'auto',
 'elegem',
 'representantes',
 'sao',
 'uns',
 'coitadosrt',
 'capitaocritica',
 'entao',
 'galera',
 'hoje',
 'que',
 'e',
 'o',
 'velorio',
 'do',
 'xbox',
 'caixinha',
 'chora',
 'e',
 'mae',
 'nao',
 've',
 'playstation5',
 'playstationshowcase',
 'httpstco…as',
 'prevendas',
 'do',
 'xbox',
 'series',
 's',
 'e',
 'xbox',
 'series',
 'x',
 'serao',
 'limitadas»',
 'httpstco8fhwitlps4',
 'httpstcono8nc30a8brt',
 'sbtonline',
 'mais',
 '5',
 'jogos',
 'vao',
 'entrar',
 'para',
 'o',
 'xbox',
 'game',
 'pass',
 'de',
 'setembro',
 'confiraduas',
 'expansoes',
 'de',
 'destiny',
 '2',
 'estao',
 'entre',
 'as',
 'novidades',
 'gra…capitaorumor',
 'vou',
 'de',
 'xbox',
 'pois',
 'no',
 'ps5',
 'alem',
 'de',
 'pagar',
 'cato',
 'no',
 'console',
 'vai',
 'ter',
 'os',
 'games',
 'tambem',
 'n

___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

___
## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou entre emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que realiza análise de sentimento e visualiza estes dados

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**